In [1]:
# Function (tools) calling

In [2]:
from devtools import debug
from dotenv import load_dotenv

%load_ext autoreload
%autoreload 2

load_dotenv()
!export PYTHONPATH=":./python"

In [3]:
from devtools import debug

from langchain_core.messages import ToolMessage
from langchain_core.tools import tool
from python.ai_core.llm import get_llm
from python.ai_core.prompts import def_prompt

### Step 1 : Ask LLM to return an instantiated function  

In [7]:
#llm = get_llm("gpt_35_edenai")
llm = get_llm("gpt_35_azure")

messages = []


@tool
def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    print("'add' tool called")
    return a + b


llm_with_tools = llm.bind_tools([add])
prompt = def_prompt("What is 11 + 11?")
chain = prompt | llm_with_tools
ai_msg = chain.invoke({})

debug(ai_msg)

2024-07-03 16:44:16.095 | INFO     | python.ai_core.llm:get_llm:405 - get LLM : gpt_35_azure - configurable: True


2024-07-03 16:44:16.484 | INFO     | python.ai_core.llm:get_configurable:361 - Cannot load gemini_pro_google: No module named 'langchain_google_vertexai'


/tmp/ipykernel_206655/2453843453.py:24 <module>
    ai_msg: AIMessage(
        content='The answer is 22.',
        response_metadata={
            'token_usage': {
                'completion_tokens': 7,
                'prompt_tokens': 72,
                'total_tokens': 79,
            },
            'model_name': 'gpt-35-turbo',
            'system_fingerprint': None,
            'finish_reason': 'stop',
            'logprobs': None,
            'content_filter_results': {},
        },
        id='run-786c866b-ab68-484c-9ace-8de22150428f-0',
        usage_metadata={
            'input_tokens': 72,
            'output_tokens': 7,
            'total_tokens': 79,
        },
    ) (AIMessage)


AIMessage(content='The answer is 22.', response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 72, 'total_tokens': 79}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}}, id='run-786c866b-ab68-484c-9ace-8de22150428f-0', usage_metadata={'input_tokens': 72, 'output_tokens': 7, 'total_tokens': 79})

In [5]:
# Step 2 : Execute the function, and send result to LLM

In [8]:
messages.append(ai_msg)

for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add}[tool_call["name"].lower()]
    tool_output = selected_tool.invoke(tool_call["args"])

    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

r = llm_with_tools.invoke(messages)

debug(r)

/tmp/ipykernel_206655/3057862763.py:11 <module>
    r: AIMessage(
        content=(
            '# Tools\n'
            '\n'
            '## functions\n'
            '\n'
            'namespace functions {\n'
            '\n'
            '// Adds a and b.\n'
            '// Args:\n'
            '// a: first int\n'
            '// b: second int\n'
            'type add = (_: {\n'
            'a: number,\n'
            'b: number,\n'
            '}) => number;\n'
            '\n'
            '} // namespace functions\n'
            '\n'
            'const add: functions.add = ({ a, b }) => a + b;\n'
            '\n'
            'console.log(add({ a: 10, b: 12 })); // Output: 22'
        ),
        response_metadata={
            'token_usage': {
                'completion_tokens': 90,
                'prompt_tokens': 70,
                'total_tokens': 160,
            },
            'model_name': 'gpt-35-turbo',
            'system_fingerprint': None,
            'finish_reason': 'stop

AIMessage(content='# Tools\n\n## functions\n\nnamespace functions {\n\n// Adds a and b.\n// Args:\n// a: first int\n// b: second int\ntype add = (_: {\na: number,\nb: number,\n}) => number;\n\n} // namespace functions\n\nconst add: functions.add = ({ a, b }) => a + b;\n\nconsole.log(add({ a: 10, b: 12 })); // Output: 22', response_metadata={'token_usage': {'completion_tokens': 90, 'prompt_tokens': 70, 'total_tokens': 160}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}}, id='run-349e674d-b904-494a-a86c-e0b4dbb39577-0', usage_metadata={'input_tokens': 70, 'output_tokens': 90, 'total_tokens': 160})

###  Other method : Use a tool calling agent

In [9]:
from langchain.agents import AgentExecutor, create_tool_calling_agent, tool
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI


@tool
def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b


# llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful Search Assistant"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

tools = [add]
agent = create_tool_calling_agent(llm, tools, prompt)  # type: ignore
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)  # type: ignore

r = agent_executor.invoke({"input": "what is 12  + 100"})
debug(r)



> Entering new AgentExecutor chain...
The result of 12 + 100 is 112.

> Finished chain.
/tmp/ipykernel_206655/1377134187.py:31 <module>
    r: {
        'input': 'what is 12  + 100',
        'output': 'The result of 12 + 100 is 112.',
    } (dict) len=2


{'input': 'what is 12  + 100', 'output': 'The result of 12 + 100 is 112.'}

In [ ]:
## Call Eden.ai provided tools